## Imports

In [1]:
import numpy as np
from numpy import sin, cos, arctan, sqrt, radians, arccos, tan, degrees
import matplotlib.pyplot as plt
import h5py, hdf5plugin
import os
import matplotlib.patches as patches
from matplotlib.patches import Rectangle, Ellipse, Circle

import glob
import time
import numba
import xarray as xr
from tqdm import tqdm

from numba import jit

%matplotlib widget

In [2]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

## Step 0: Crystal Parameters/Diffraction Geometry Defintions/Coordination Matrix Defintion
* Only done once

In [3]:
#### Lattice Parameters ####
#Ru-0001 HCP

a = 2.71 #Angstrom
b = a
c = 4.27

#### Detector Parameters ####

det_xrange = 1475
det_yrange = 1679
pixelsize = 172e-6 #m
cpx = 727.3 #cen pixel x
cpy = 1624.45 #cen pixel y

#### Experiment Parameters ####

E = 75 #keV
SDD = 0.7781 #m
mu = 0.08 #degrees
lam = 0.1653

ko = (2*np.pi)/lam

## Step 1: Calculation of "Bulk" Arrays (Qx, Qy, Qz, TwoTheta, Chi, Correction Factors)
* Only done once
* Everything that is a function of delta and gamma

### Calculation and Saving of Bulk Array (Commented out)

In [4]:
# #### Creating 2D array for values that are fnc of x, y pixel ####
# #Typical time = ~40 seconds

# ###### Initization of Arrays ######

# #Q
# qx_2d = np.zeros((det_yrange, det_xrange))
# qy_2d = np.zeros((det_yrange, det_xrange))
# qz_2d = np.zeros((det_yrange, det_xrange))

# #Powder Rings
# twotheta_2d = np.zeros((det_yrange, det_xrange))
# chi_2d = np.zeros((det_yrange, det_xrange))

# #Powder Rings
# delta_2d = np.zeros((det_yrange, det_xrange))
# gamma_2d = np.zeros((det_yrange, det_xrange))

# #Correction Factors
# L_2d = np.zeros((det_yrange, det_xrange)) #Lorentz Correction
# Polar_2d = np.zeros((det_yrange, det_xrange)) #Polarization Factor
# Crod_2d = np.zeros((det_yrange, det_xrange)) #Rod Correction
# Carea_2d = np.zeros((det_yrange, det_xrange)) #Detector Correction i (?)

# #Masking Array
# distance = np.zeros((det_yrange, det_xrange)) #For Masking Secondary Diffraction

# images_norm = []
# total_iterations = det_xrange
# progress_bar = tqdm(total=total_iterations, desc="Processing")
# for pixel_x in range(det_xrange):
#     progress_bar.update(1)
    
#     delta_px = int(pixel_x - cpx)
#     delta_x = delta_px * pixelsize
#     gamma = arctan(delta_x / SDD)

#     for pixel_y in range(det_yrange):
#         delta_py = int(cpy - pixel_y)
#         delta_y = delta_py * pixelsize

#         delta = arctan(delta_y / (sqrt(SDD**2 + delta_x**2)) - (radians(mu) * cos(gamma)))

#         gamma_2d[pixel_y, pixel_x] = gamma
#         delta_2d[pixel_y, pixel_x] = delta

#         qx_2d[pixel_y, pixel_x] = np.abs(ko * ((cos(delta) * cos(gamma)) - cos(radians(mu))))
#         qy_2d[pixel_y, pixel_x] = ko * (cos(delta) * sin(gamma))
#         qz_2d[pixel_y, pixel_x] = ko * (sin(delta)+sin(radians(mu)))

#         #Correction Factors
#         L_2d[pixel_y, pixel_x] = np.abs((cos(radians(0))*cos(delta)*sin(gamma))) #Lorentz
#         Polar_2d[pixel_y, pixel_x] = 1 - (sin(radians(mu)) * cos(gamma) * cos(delta) + cos(radians(mu))*sin(delta))**2 #Polarization Vlieg
#         # Polar_2d[pixel_y, pixel_x] = (cos(delta))**2 * (sin(gamma))**2 + (sin(delta))**2 #Polarization HAT
#         Crod_2d[pixel_y, pixel_x] = 2*cos(delta)/(cos(radians(mu))+cos(radians(mu))+sin(2*radians(mu))*sin(delta)+(2*(sin(radians(mu))**2)*cos(gamma)*cos(delta))) #Rod Correction
#         Carea_2d[pixel_y, pixel_x] = 1/(sin(gamma)*cos(radians(mu))) #Area Correction
         
#         #Calculate twotehta - chi image projection for background subtraction
#         twotheta_2d[pixel_y, pixel_x] = degrees(arccos(cos(gamma)*cos(delta)))
#         if delta_y == 0 and delta_x >= 0:
#             chi_2d[pixel_y, pixel_x] = 90
#         if delta_y == 0 and delta_x <= 0:
#             chi_2d[pixel_y, pixel_x] = -90
#         if delta_y == 0:
#             chi_2d[pixel_y, pixel_x] = 90
#         else:
#             chi_2d[pixel_y, pixel_x] = degrees(arctan(delta_x/delta_y))
            
#         # Distance Array for Masking of Secondary Scattering
        
#         #Parameters for getting rid of secondary scattering
#         center_x_ss1, center_y_ss1 = 1047, 1454
        
#         distance[pixel_y, pixel_x] = (np.sqrt((pixel_x - center_x_ss1)**2 + (pixel_y - center_y_ss1)**2))
        
        
# progress_bar.close()

In [5]:
# delta_2d_array = np.array(delta_2d)
# np.max(delta_2d_array)

In [6]:
# #Save Everything to make life easier going forward
# #Q_Space
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\qx_2d', qx_2d)
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\qy_2d', qy_2d)
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\qz_2d', qz_2d)

# #TwoTheta/Chi and distance for Masking
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\twotheta_2d', twotheta_2d)
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\chi_2d', chi_2d)
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\distance', distance)

# #Correction Factors
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\L_2d', L_2d)
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\Polar_2d', Polar_2d)
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\Crod_2d', Crod_2d)
# np.savetxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\Carea_2d', Carea_2d)

### Bringing in Saved Bulk Arrays

In [7]:
#Q-Space Arrays
qx_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\qx_2d')
qy_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\qy_2d')
qz_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\qz_2d')

#Masking Arrays
twotheta_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\twotheta_2d')
chi_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\chi_2d')
distance = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\distance')

#Correction Factors
L_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\L_2d')
Polar_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\Polar_2d')
Crod_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\Crod_2d')
Carea_2d = np.loadtxt(r'D:\CTR_MyDrive\Extracted_BulkArrays\Carea_2d')

In [8]:
# plt.close('all')
# plt.imshow(Crod_2d)
# plt.colorbar()
# plt.title('Lorentz')
# plt.show()

## Step 2: Path Finding to HDF5 File
* Ensure this is run before going to step 3

In [9]:
sample_list = ['Ru1_Pristine', 'Ru1_Hexane', 'Ru1_Pristine2', 'Ru1_Hexane2',
               'Ru1_Hexane3', 'Ru2_Hexene', 'Ru1_iodoethane', 'Ru1_iodoethane_cont',
               'Ru2_Pristine1', 'Ru1_pristine3', 'PEEK_bkg', 'Ru2_DMB', 'Ru2_DMB_PnB', 
               'Ru2_FBenz', 'Ru1_Pristine4', 'Ru1_DMB'] #All Samples for experiment ma5870


######### Select Sample and Scan Number ##########
sample = sample_list[11]
scanNo = '146'

print(color.BOLD + 'Sample Selected:' + color.END, sample)
print(color.BOLD + 'Scan Selected:' + color.END, scanNo) 

#Path-finding to H5
path = r'D:\ma5870\id31\20231207\RAW_DATA'

sampledir = sample+'_0001'

directory = os.path.join(path, sample, sampledir, sampledir)

file = h5py.File(directory+'.h5', 'r')

Sample Selected: Ru2_DMB
Scan Selected: 146


In [10]:
# ##Print the scan in Qspace?

# ############### Bringing in the images and mondio ###################
# theta = file[str(scanNo)+'.1/measurement/th']
# images = file[str(scanNo)+'.1/measurement/p3']
# mondio = file[str(scanNo)+'.1/measurement/mondio']

# images_norm = []

# for image in range(min(len(mondio), len(images))):
              
#     images_norm.append(images[image, :, :]/mondio[image])
               
# images_norm = np.array(images_norm)

# stacked_images = np.stack(images_norm, axis=-1)
# imagesum = np.maximum.reduce(stacked_images, axis=-1) # This is actually a max intensity image

# imagesum_flat = imagesum.flatten()
# qx_flat = qx_2d.flatten()
# qy_flat = qy_2d.flatten()
# qz_flat = qz_2d.flatten()

# qr_flat = np.sqrt(qx_flat**2+qy_flat**2+qx_flat*qy_flat)

# plt.close('all')
# plt.title('Ru1_Pristine', fontsize = 14)
# plt.scatter(qr_flat,qz_flat, c = np.log(imagesum_flat))
# plt.xlabel('Q$_r$',fontsize = 12)
# plt.ylabel('Q$_z$',fontsize = 12)
# plt.show()

## Step 3: Extraction of Powder-Rings

### Testing (Normally Commented Out)

#### - Masking Secondary Scattering Rings

In [11]:
# ######################## Testing Code (Use to ensure everything working ################################

# theta = file[str(scanNo)+'.1/measurement/th']
# images = file[str(scanNo)+'.1/measurement/p3']
# mondio = file[str(scanNo)+'.1/measurement/mondio']

# #Range for removing specific th_values
# # #Ru1_Pristine
# # ss_min = 37
# # ss_max = 39

# #Ru1_DMB
# ss_min = 4.8
# ss_max = 5.1

# plt.close('all')

# #Parameters for getting rid of secondary scattering
# center_x, center_y = 1047, 1454
# rad1 = 250
# rad2 = 150

# images_masked_short = []
# images_og_short = []

# total_iterations = min(len(mondio), len(images))
# progress_bar = tqdm(total=total_iterations, desc="Processing")

# distance = np.zeros((det_yrange, det_xrange)) #Lorentz Correction

# for x in range(det_xrange):
#     for y in range(det_yrange):
#         distance[y, x] = (np.sqrt((x - center_x)**2 + (y - center_y)**2))

# for image in range(min(len(mondio), len(images))):
#     progress_bar.update(1)
#     if ss_min <= theta[image] <= ss_max:
#         progress_bar.update(1)
#         masked_image = images[image, :, :].copy()
#         og_image = images[image, :, :].copy()
        
#         mask = (distance > rad2) & (distance <= rad1)
        
#         masked_image[mask] = 0
        
#         images_masked_short.append(masked_image/mondio[image])
#         images_og_short.append(og_image/mondio[image])
    
# progress_bar.close()

# images_masked_short = np.array(images_masked_short)

# stacked_mask_short = np.stack(images_masked_short, axis = -1)
# imagesum_mask_short = np.maximum.reduce(stacked_mask_short, axis = -1)

# images_og_short = np.array(images_og_short)

# stacked_og_short = np.stack(images_og_short, axis = -1)
# imagesum_og_short = np.maximum.reduce(stacked_og_short, axis = -1)

# ############# Plotting #################

# plt.close('all')
# fig, axs = plt.subplots(1, 2, figsize=(20, 8))

# # Plot the masked image
# im1 = axs[0].imshow(np.log10(imagesum_og_short), cmap='jet', vmax=-3.5, vmin=-5)
# outer_circle = Circle((center_x, center_y), rad1, color='red', linestyle='--', fill=False, linewidth=2)
# axs[0].add_patch(outer_circle)
# inner_circle = Circle((center_x, center_y), rad2, color='red', linestyle='--', fill=False, linewidth=2)
# axs[0].add_patch(inner_circle)
# axs[0].set_xlabel('x', fontsize = 14)
# axs[0].set_ylabel('y', fontsize = 14)
# axs[0].tick_params(axis='x', labelsize=12)
# axs[0].tick_params(axis='y', labelsize=12)
# axs[0].set_title('Original Image', fontsize = 14)
# cbar1 = plt.colorbar(im1, ax=axs[0])
# cbar1.set_label('Log10(Intensity)', fontsize=14)

# # Plot the original image
# im2 = axs[1].imshow(np.log10(imagesum_mask_short), cmap='jet', vmax=-3.5, vmin=-5)
# axs[1].set_xlabel('x', fontsize = 14)
# axs[1].set_ylabel('y', fontsize = 14)
# axs[1].tick_params(axis='x', labelsize=12)
# axs[1].tick_params(axis='y', labelsize=12)
# axs[1].set_title('Masked Image', fontsize = 18)
# cbar2 = plt.colorbar(im2, ax=axs[1])
# cbar2.set_label('Log10(Intensity)', fontsize=14)

# # Adjust layout and show the plot
# plt.tight_layout()
# plt.show()

#### Masking Normal Powder Diffractions with 2Theta/Chi Plots

In [12]:
# #### Visualization of Powder Ring Subtraction --> Cannot use this for B-Matrix and Theta calculation and must final cell in step 2 ###### 

# twotheta_flat = twotheta_2d.flatten()
# chi_flat = chi_2d.flatten()
# imagesum_masked_short_flat = imagesum_mask_short.flatten()

# #Define Mask Region for General PEEK rings
# twotheta_min, twotheta_max = 1.95, 3.1
# chi_min, chi_max = -90, 90

# #Define Mask Region for Mask2
# twotheta2_min, twotheta2_max = 3.8, 4.7
# chi2_min, chi2_max = -55, 57

# twotheta3_min, twotheta3_max = 4.7, 5.2
# chi3_min, chi3_max = -45, 45

# # twotheta4_min, twotheta4_max = 3.1, 5.5
# # chi4_min, chi4_max = -25, 25


# #### Plotting ####
# plt.close('all')
# fig, axs = plt.subplots(1,2, figsize = (20,10))

# # Original Image with Powder-Ring Regions Indicated #
# scatter1 = axs[0].scatter(twotheta_flat, chi_flat, c=np.log10(imagesum_og_short), cmap='jet', marker='.')

# #Mask 1 Plot
# rect1 = Rectangle((twotheta_min, chi_min), twotheta_max - twotheta_min, chi_max - chi_min, linewidth=2, edgecolor='r', facecolor='none', linestyle='--')
# axs[0].add_patch(rect1)

# #Mask 2 Plot
# rect2 = Rectangle((twotheta2_min, chi2_min), twotheta2_max - twotheta2_min, chi2_max - chi2_min, linewidth=2, edgecolor='r', facecolor='none', linestyle='--')
# axs[0].add_patch(rect2)

# rect3 = Rectangle((twotheta3_min, chi3_min), twotheta3_max - twotheta3_min, chi3_max - chi3_min, linewidth=2, edgecolor='r', facecolor='none', linestyle='--')
# axs[0].add_patch(rect3)

# axs[0].set_xlabel('2$\Theta$')
# axs[0].set_ylabel('$\chi$')
# axs[0].set_title('Original Data with Mask Region Indicated')
# fig.colorbar(scatter1, ax=axs[0], label='Log10(Intensity)')

# # Perform masking
# mask = (chi_2d >= chi_min) & (chi_2d <= chi_max) & (twotheta_2d >= twotheta_min) & (twotheta_2d <= twotheta_max)
# mask2 = (chi_2d >= chi2_min) & (chi_2d <= chi2_max) & (twotheta_2d >= twotheta2_min) & (twotheta_2d <= twotheta2_max)
# mask3 = (chi_2d >= chi3_min) & (chi_2d <= chi3_max) & (twotheta_2d >= twotheta3_min) & (twotheta_2d <= twotheta3_max)
# # mask4 = (chi_2d >= chi4_min) & (chi_2d <= chi4_max) & (twotheta_2d >= twotheta4_min) & (twotheta_2d <= twotheta4_max)

# # Combine all masks
# combined_mask = mask | mask2 | mask3 #| mask4

# # Apply the combined mask to the data
# imagesum_mask_short[combined_mask] = 0
# intensity_mask_flat = imagesum_mask_short.flatten()

# # New Image with Powder-Rings Masked Out #
# scatter2 = axs[1].scatter(twotheta_flat, chi_flat, c=np.log10(intensity_mask_flat*10), cmap='jet', marker='.')
# axs[1].set_xlabel('2$\Theta$')
# axs[1].set_ylabel('$\chi$')
# axs[1].set_title('Masked Data')
# fig.colorbar(scatter2, ax=axs[1], label='Log10(Intensity)')

# # Show the plot
# plt.tight_layout()
# plt.show()

### Application to all Thetas + Addition of PEEK BKG Subtraction

In [13]:
# progress_bar.close()
############### Bringing in the images and mondio ###################
theta = file[str(scanNo)+'.1/measurement/th']
images = file[str(scanNo)+'.1/measurement/p3']
mondio = file[str(scanNo)+'.1/measurement/mondio']

# ############## Define Mask Region for General PEEK rings ###########
# twotheta_min, twotheta_max = 1.95, 3.1
# chi_min, chi_max = -90, 90

# # Define Mask Region for Mask2
# twotheta2_min, twotheta2_max = 3.8, 4.7
# chi2_min, chi2_max = -55, 55

# twotheta3_min, twotheta3_max = 4.7, 5.2
# chi3_min, chi3_max = -49.5, 49.5

#Define Mask Region for General PEEK rings
twotheta_min, twotheta_max = 1.95, 3.1
chi_min, chi_max = -90, 90

#Define Mask Region for Mask2
twotheta2_min, twotheta2_max = 3.8, 4.7
chi2_min, chi2_max = -55, 57

twotheta3_min, twotheta3_max = 4.7, 5.2
chi3_min, chi3_max = -45, 45

# twotheta4_min, twotheta4_max = 3.1, 5.5
# chi4_min, chi4_max = -90, 25

############3# Define Mask Region for Secondary Scattering ############
rad1 = 250
rad2 = 150

# #Range for removing specific th_values
# #Ru1_Pristine
# ss_min = 38
# ss_max = 39

# #Ru1_DMB
# ss_min = 4.8
# ss_max = 5.1

# #Ru1_Pristine4
# ss_min = 5
# ss_max = 5.57

# #Ru2_Pristine1
# ss_min = 20
# ss_max = 21

#Ru2_DMB
ss_min = 19.81
ss_max = 20.36

# #Ru2_Hexene
# ss_min = 31.53
# ss_max = 32.07

# #Ru1_Pristine3
# ss_min = 13.20
# ss_max = 13.47


plt.close('all')

################ PEEK BKG ###################

######### Switch to PEEK BKG Scan ##########
sample = sample_list[10]
scanNo_bkg = '1'

print(color.BOLD + 'Sample for BKG Selected:' + color.END, sample)

#Path-finding to H5
path = r'D:\ma5870\id31\20231207\RAW_DATA'

sampledir = sample+'_0001'

directory = os.path.join(path, sample, sampledir, sampledir)

file = h5py.File(directory+'.h5', 'r')

images_bkg = file[str(scanNo_bkg)+'.1/measurement/p3']
mondio_bkg = file[str(scanNo_bkg)+'.1/measurement/mondio']

images_norm_sub = []
images_norm = []

##### Looping Through all Thetas and Applying Masks w/ BKG Sub at end ###########

total_iterations = min(len(mondio), len(images))

progress_bar = tqdm(total=total_iterations, desc="Processing")

############# Change Theta To Reduce Memory Requirement ###############

######################################################
###### Ru1_Pristine

# #Range for 100-103
# th_min = 34
# th_max = 40

# #Range for 100-106
# th_min = 17
# th_max = 40

# #Range for 105-107
# th_min = 9
# th_max = 23

# # Range for 103-105
# th_min = 23
# th_max = 33

#Range for 110-115
# th_min = 2
# th_max = 8

########################################################
###### Ru1_DMB

# #Range for 100-103
# th_min = 0
# th_max = 6.5

# #Range for 103-105
# th_min = -9.5
# th_max = 0

# #Range for 105-107
# th_min = -30
# th_max = -9

########################################################
###### Ru1_Pristine4

# #Range for 100-103
# th_min = 0
# th_max = 6.8

# #Range for 105-107
# th_min = -26
# th_max = -8.5

# #Range for 103-105
# th_min = -8.8
# th_max = 0

##########################################################
###### Ru2_Pristine1

# #Range for 100-103
# th_min = 16
# th_max = 22

# #Range for 105-107
# th_min = -9
# th_max = 8

# #Range for 103-105
# th_min = 6.82
# th_max = 16

#############################################################
###### Ru2_DMB

# #Range for 100-103
# th_min = 15
# th_max = 22

# #Range for 105-107
# th_min = -10
# th_max = 6.5

#Range for 103-105
th_min = 6
th_max = 15

#########################################################
###### Ru2_Hexene

# #Range for 100-103
# th_min = 27
# th_max = 33

# #Range for 105-107
# th_min = 3
# th_max = 18

# #Range for 103-105
# th_min = 18
# th_max = 27

########################################################
###### Ru1_Pristine3

# #Range for 100-103
# th_min = 8
# th_max = 15

# #Range for 105-107
# th_min = -16
# th_max = -1.2

# #Range for 103-105
# th_min = -1.2
# th_max = 8.5


th_range = []

for image in range(min(len(mondio), len(images))):
    if th_min <= int(theta[image]) <= th_max:
        th_range.append(radians(theta[image]))
        progress_bar.update(1)

        norm_image = images[image] / mondio[image]
        norm_bkg = images_bkg[image]/ mondio_bkg[image]

        mask = (chi_2d >= chi_min) & (chi_2d <= chi_max) & (twotheta_2d >= twotheta_min) & (twotheta_2d <= twotheta_max)
        mask2 = (chi_2d >= chi2_min) & (chi_2d <= chi2_max) & (twotheta_2d >= twotheta2_min) & (twotheta_2d <= twotheta2_max)
        mask3 = (chi_2d >= chi3_min) & (chi_2d <= chi3_max) & (twotheta_2d >= twotheta3_min) & (twotheta_2d <= twotheta3_max)
        # mask4 = (chi_2d >= chi4_min) & (chi_2d <= chi4_max) & (twotheta_2d >= twotheta4_min) & (twotheta_2d <= twotheta4_max)
        

        # Combine all masks
        combined_mask = mask | mask2 | mask3 #| mask4

        #Apply masks
        norm_image[combined_mask] = 0
        norm_bkg[combined_mask] = 0 
    
        if ss_min <= theta[image] <= ss_max:
            mask_ss = (distance >= rad2) & (distance <= rad1)

            norm_image[mask_ss] = 0
            norm_bkg[mask_ss] = 0

        image_sub = norm_image - norm_bkg

        images_norm_sub.append(image_sub)
        images_norm.append(norm_image)

progress_bar.close()

Sample for BKG Selected: PEEK_bkg


Processing:   8%|█████▎                                                             | 144/1801 [00:11<02:07, 13.03it/s]


### Redirect and Plot Image

In [14]:
########### Redirect back to Sample to Interest ##############
######### Select Sample and Scan Number ##########
sample = sample_list[11]
scanNo = '146'

print(color.BOLD + 'Sample Selected:' + color.END, sample)
print(color.BOLD + 'Scan Selected:' + color.END, scanNo) 

#Path-finding to H5
path = r'D:\ma5870\id31\20231207\RAW_DATA'

sampledir = sample+'_0001'

directory = os.path.join(path, sample, sampledir, sampledir)

file = h5py.File(directory+'.h5', 'r')

theta = file[str(scanNo)+'.1/measurement/th']
images = file[str(scanNo)+'.1/measurement/p3']
mondio = file[str(scanNo)+'.1/measurement/mondio']

Sample Selected: Ru2_DMB
Scan Selected: 146


In [15]:
# import dask.array as da
# images_dask = [da.from_array(img, chunks="auto") for img in images_norm]
# stacked_images = da.stack(images_dask, axis=-1)

# # Do you want to plot the full image?
# # If this is run you cannot do the powder ring diffraction subtraction and full kernel will need to be restarted

# plt.imshow(np.log10(subset.compute()), vmin=-6, vmax=-4)

## Step 4: B-Matrix Transformation and Crop to Theta Range and Window (Rod Pixel Range) of Interest 
* User defines pixel associated with Bragg Reflections (101 and 112) to set Orientation Matrix (once)
* User defines theta corresponding to Bragg Reflections to set Oreintation Matrix (every time)

In [16]:
theta = file[str(scanNo)+'.1/measurement/th']

# # #Ru1_Pristine
# bragg_101_th = 38.775
# bragg_112_th = 6.525

#Ru1_DMB
# bragg_101_th = 4.91
# # bragg_112_th = 32.6185
# bragg_112_th = 32.619

# # #Ru1_Pristine4
# bragg_101_th = 5.3265
# bragg_112_th = 33.1045

# #Ru2_Pristine1
# bragg_101_th = 20.5355
# bragg_112_th = 48.3135

#Ru2_DMB
bragg_101_th = 19.98
# # bragg_101_th = 19.9795
bragg_112_th = 47.827

# Ru2_Hexene
# bragg_101_th = 31.9665 #25, 100
# bragg_112_th = 59.4335
# bragg_101_th = 31.901 #50
# bragg_112_th = 59.368
# bragg_101_th = 31.634 #150 and 175
# bragg_112_th = 59.3675

# #Ru1_Pristine3
# bragg_101_th = 13.3005
# bragg_112_th = 40.9675


#This should be constant no matter what the the alignment is
pix_x_101 = 1048
pix_y_101 = 1448

pix_x_112 = 1286
pix_y_112 = 1271


for image in range(min(len(mondio), len(images))):
    if theta[image] == bragg_101_th:
        th_101 = theta[image]
        print(color.BOLD + 'Theta for 101:' + color.END, th_101)

    elif theta[image] == bragg_112_th:
        th_112 = theta[image]
        print(color.BOLD + 'Theta for 112:'+ color.END, th_112)

########

#Qx Qy for 101 Bragg
qxphi_101 = qy_2d[pix_y_101, pix_x_101]* sin(radians(th_101)) + qx_2d[pix_y_101, pix_x_101]* cos(radians(th_101))
qyphi_101 = qy_2d[pix_y_101, pix_x_101]* cos(radians(th_101)) - qx_2d[pix_y_101, pix_x_101]* sin(radians(th_101))
qzphi_101 = qz_2d[pix_y_101, pix_x_101]

#Qx Qy for 112 Bragg
qxphi_112 = qy_2d[pix_y_112, pix_x_112]* sin(radians(th_101)) + qx_2d[pix_y_112, pix_x_112]* cos(radians(th_112))
qyphi_112 = qy_2d[pix_y_112, pix_x_112]* cos(radians(th_101)) - qx_2d[pix_y_112, pix_x_112]* sin(radians(th_112))
qzphi_112 = qz_2d[pix_y_112, pix_x_112]

#########
q_101 = [qxphi_101, qyphi_101, qzphi_101]
q_112 = [qxphi_112, qyphi_112, qzphi_112]

hkl_101 = [1, 0, 1]
hkl_112 = [1, 1, 2]

A1 = q_101[0]/hkl_101[0]
A2 = (q_112[0] - A1*hkl_112[0])/hkl_112[0]

B1 = q_101[1]/hkl_112[1]
B2 = (q_112[1] - B1*hkl_112[1])/hkl_112[1]

B = np.array([[A1, A2, 0],
              [B1, B2, 0],
              [0, 0, 2*np.pi/c]])

Theta for 101: 19.98
Theta for 112: 47.827


In [17]:
A1, A2, B1, B2

(np.float64(1.0297127167748268),
 np.float64(0.8205434831313669),
 np.float64(2.477438028508875),
 np.float64(1.5907602503320066))

In [18]:
## User-Defined Rod Parameters ##
theta = file[str(scanNo)+'.1/measurement/th']

########### Change HK ################
#Range For 10L
pix_x_max = 1060
pix_x_min = 1020

# #For 11L
# pix_x_max = 1300
# pix_x_min = 1260

############# Change L ################
# # Range for 100-103
# pix_y_max = 1609
# pix_y_min = 1100

# # Range for 103-105
pix_y_max = 1100
pix_y_min = 720

# #Range for 105-107
# pix_y_max = 730
# pix_y_min = 360

#Range for 110-115
# pix_y_max = 1609
# pix_y_min = 900

# # Range for 100-106
# pix_y_max = 1609
# pix_y_min = 540

I_3d = []

for image in range(len(images_norm)):
    I_3d.append((images_norm[image][pix_y_min : pix_y_max+1, pix_x_min : pix_x_max+1]))
    
I_3d_array = np.array(I_3d)

## Cropping 2d Matrices Defined in Step 1 to be for search area ##
qx_2d_rod = qx_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
qy_2d_rod = qy_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
qz_2d_rod = qz_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]

L_2d_rod = L_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
Crod_2d_rod = Crod_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
Carea_2d_rod = Carea_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
Polar_2d_rod = Polar_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]

## Transformation into 3D Q-Space ##
qx_3d = []
qy_3d = []
qz_3d = []

for theta in th_range:
    qx = (qy_2d_rod * sin(theta) + qx_2d_rod * cos(theta))
    qx_3d.append(qx)

    qy = (qy_2d_rod * cos(theta) - qx_2d_rod * sin(theta))
    qy_3d.append(qy)

    qz = qz_2d_rod
    qz_3d.append(qz)

qx_3d_array = np.array(qx_3d)
qy_3d_array = np.array(qy_3d)
qz_3d_array = np.array(qz_3d)

qx_flat = qx_3d_array.flatten()
qy_flat = qy_3d_array.flatten()
qz_flat = qz_3d_array.flatten()

## Application of Oreintation Matrix to each coordinate qx,qy,qz ##

h = []
k = []
l = []

total_iterations = len(qx_flat)

progress_bar = tqdm(total=total_iterations, desc="Processing")

for x, y, z in zip(qx_flat, qy_flat, qz_flat):
    progress_bar.update(1)
    combined_array = [x, y, z]
    # print(combined_array)
    result_array = np.linalg.solve(B, combined_array)
    h.append(result_array[0])
    k.append(result_array[1])
    l.append(result_array[2])
    
progress_bar.close()

## Cropping 2d Correction Matrices Defined in Step 1 to be for search area ##
L_2d_rod = L_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
Polar_2d_rod = Polar_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
Crod_2d_rod = Crod_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]
Carea_2d_rod = Carea_2d[pix_y_min : pix_y_max+1, pix_x_min : pix_x_max +1]

## Extraction of Structure Factor from Intensity ##
count_time = 0.2 #seconds
re = 2.8179e-5 #Angstrom
Area = 50000**2 #Angstrom^2 --> assuming a 5 micron x 5 micron beam
Au = (3*sqrt(3) / 2) * a**2 #SA of Ru Unit cell in 0001 plane

Corr_total =  count_time * (Area / (Au**2)) * re**2 * lam**2 * Polar_2d_rod * (L_2d_rod)**-1 * Crod_2d_rod *  Carea_2d_rod

SF_3d_array = I_3d_array / Corr_total

Processing: 100%|████████████████████████████████████████████████████████| 2249424/2249424 [00:21<00:00, 103267.20it/s]


## Step 5: Plotting the #D Structure Factor or Intensity

In [19]:
SF_flat = SF_3d_array.flatten()
I_flat = I_3d_array.flatten()

# plt.close('all')

# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(111, projection='3d')

# # Scatter plot with color and size based on images_array

# scatter = ax.scatter(qx_flat, qy_flat, l, c=np.log10(I_flat), cmap='jet',marker = 'o', alpha = 0.002)

# # Add colorbar
# # cbar = plt.colorbar()

# # Set axis labels
# ax.set_xlabel('qx')
# ax.set_ylabel('qy')
# ax.set_zlabel('qz')

# ax.view_init(0, 0)

# plt.show()

## Step 5: Save Data

In [20]:
date = '10.15.2025_'
h_val = 1
k_val = 0

min_l = 3
max_l = 5

temp = 252

path_h = r'D:\CTR_MyDrive\Extracted_3D'
path_sample = str(sample)
path_rodrange = str(h_val)+str(k_val)+str(min_l)+'-'+str(h_val)+str(k_val)+str(max_l)
path_filename = date + str(sample) + '_' + str(temp)

full_path_h = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_h_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'
full_path_k = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_k_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'
full_path_l = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_l_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'
full_path_SF = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_SF_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'

full_path_Qx = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_Qx_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'
full_path_Qy = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_Qy_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'
full_path_Qz = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_Qz_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'

full_path_I = path_h + '//' + path_sample + '//' + path_rodrange + '//' + path_filename + 'C_I_' + str(h_val)+str(k_val)+str(min_l)+'_'+str(h_val)+str(k_val)+str(max_l) + '.txt'

np.savetxt(full_path_h, h)
np.savetxt(full_path_k, k)
np.savetxt(full_path_l, l)
np.savetxt(full_path_SF, SF_flat)

np.savetxt(full_path_Qx, qx_flat)
np.savetxt(full_path_Qy, qy_flat)
np.savetxt(full_path_Qz, qz_flat)

np.savetxt(full_path_I, I_flat)

print('You extracted the ROD in 3D Space!! #NiceRod')

You extracted the ROD in 3D Space!! #NiceRod
